# Import Necessary Libraries

In [ ]:
import json
import requests
import sqlite3
import pandas as pd
import numpy as np
import pymongo
import datetime

In [ ]:
#Provide Api Key Code
def get_keys(path):
    with open(path) as f:
        return json.load(f)
keys = get_keys("/Users/Lucas/.secret/API/darksky_api.json")
api_key = keys['api_key']

# Import Weather Data

In [ ]:
class WeatherGetter(object):
    def __init__(self):
        self.secret_key = keys['api_key']
        self.berlin_lat = "52.5200"
        self.berlin_long = "13.4050"
        self.url_base = "https://api.darksky.net/forecast"
        self.exclude = 'currently,flags,minutely,hourly,alerts'
        
    def get_weather_forcast(self, datetime_string):
        year, month, day = self.format_datetime(datetime_string)
        
        datetime = "{}-{}-{}T12:00:00".format(year, month, day)
        full_url = "{}/{}/{},{},{}?exclude={}".format(self.url_base, self.secret_key, 
                                                     self.berlin_lat, self.berlin_long, 
                                                     datetime, self.exclude)
        response = requests.get(full_url)
        if response.status_code == 200: 
            print(response.status_code)
        else:
            print('Error: The API call failed')
        return response

    def get_rain(self, response):
        data = json.loads(response.text)
        daily = data['daily']
        data =  daily['data']
        data = data[0]
        try:
            if data['icon'] == 'rain':
                print(data['icon'])
                return True
            else: 
                print(data['icon'])
                return False
        except KeyError:
            print('no rain')
            return False
            
    def format_datetime(self,datetime_string):
        year = datetime_string[:4]
        month = datetime_string[5:7]
        day = datetime_string[8:]
        
        return year, month, day
    
    def was_raining(self, datetime_string):
        response = self.get_weather_forcast(datetime_string)
        itrain = self.get_rain(response)
        return itrain
    
    def all_weather(self, date_list):
        weather = {}
        
        for date in date_list:
            weather[date] = self.was_raining(date)
        return weather

In [ ]:
#test Function

wth = WeatherGetter()
wth.get_weather_forcast('2011-07-15')

# Import and Structure Data from Kaggle

In [ ]:
conn = sqlite3.connect('''database.sqlite''')
c = conn.cursor()

#get Match Data
c.execute("""SELECT * from matches where season = 2011 ORDER BY Date ASC""")
full_data = pd.DataFrame(c.fetchall())
full_data.columns = [i[0] for i in c.description]
full_data.head()

In [ ]:
#Get Team Values
c.execute("""SELECT * from teams where season = 2011""")
teams = pd.DataFrame(c.fetchall())
teams.columns = [i[0] for i in c.description]
teams.head()

In [ ]:
#Get forecast for game dates 
date_list = list(full_data.Date.unique())
forecast = wth.all_weather(date_list)
rain = []
for date in full_data.Date:
    rain.append(forecast[date])

In [ ]:
full_data['Rain'] = rain
full_data.head()

In [ ]:
# Get List of Teams
unique_teams = full_data['HomeTeam'].unique()

In [ ]:
#Prepare Data for Mongo Database
def team_record(team_names):
    team_data = {}
    for team in unique_teams:
        data = {'total_matches': int(len(full_data.query("HomeTeam == @team or AwayTeam == @team"))) , 
                'total_wins_2011': int(len(full_data.query("(HomeTeam == @team & FTR == 'H') | (AwayTeam == @team & FTR == 'A')"))),
                'total_losses_2011': int(len(full_data.query("(HomeTeam == @team & FTR != 'H')|(AwayTeam == @team & FTR != 'A')"))),
                'total_rain_games_2011': int(len(full_data.query("(HomeTeam == @team or AwayTeam == @team) & (Rain == True)"))),
                'rain_wins_2011': int(len(full_data.query("(HomeTeam == @team & FTR == 'H' & Rain == True) | (AwayTeam == @team & FTR == 'A' & Rain == True)"))),
                'rain_losses_2011': int(len(full_data.query("(HomeTeam == @team & FTR != 'H' & Rain == True) | (AwayTeam == @team & FTR != 'A' & Rain == True)"))),
                'total_goals_2011': int(full_data.query("HomeTeam == @team")['FTHG'].sum() + full_data.query("AwayTeam == @team")['FTAG'].sum())}
        team_data[team] = data
    
    for team in unique_teams:
        team_data[team]['win_percentage'] = round(team_data[team]['total_wins_2011']/(team_data[team]['total_wins_2011']+team_data[team]['total_losses_2011']),2)
        team_data[team]['rain_win_percentage'] = round(team_data[team]['rain_wins_2011']/(team_data[team]['rain_wins_2011']+team_data[team]['rain_losses_2011']),2)
    for team in unique_teams:
        try:
            team_data[team]['team_market_value'] = int(teams.query("TeamName == @team")['OverallMarketValueHome'].values)
        except:
            team_data[team]['team_market_value'] = 'NaN'
    
    return team_data

In [ ]:
team_data = team_record(unique_teams)
team_data

# Initiate Mongo Server

In [ ]:
class MongodbHandler(object):
    
    def __init__(self):
        self.myclient = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
        self.db = self.myclient['summative_lab']
        self.coll = self.db['summative_collection']
    
    def data_format(self, team_name, market_value, total_games, total_goals, win_percentage, rain_win_percentage, all_viz):
        data = {"team_name": team_name,
                "Market_Value": market_value,
                "games_playes_2011": total_games,
                "goals_2011": total_goals,
                "win_percentage_2011": win_percentage,
                "rain_win_percentage_2011": rain_win_percentage,
                "win_rate_histo": all_viz}
        return data
    
    def insert_record(self, record):
        return self.coll.insert_one(data)

# Get Visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def get_all_viz(team, show_viz=False):
    wins_value_all = team['total_wins_2011']
    losses_value_all = team['total_losses_2011']
    x_all = ['wins','losses']
    y_all = [wins_value_all,losses_value_all]
    bar_all = plt.bar(x_all,y_all)
    return bar_all

# def get_rain_viz(team, show_viz=False):    
#     wins_value_rain = team['rain_wins_2011']
#     losses_value_rain = team['rain_losses_2011']
#     x_rain = ['wins','losses']
#     y_rain = [wins_value_rain,losses_value_rain]
#     bar_rain = plt.bar(x_rain,y_rain)
#     return bar_rain
print(get_all_viz(team_data['Nurnberg']))

# Upload to Mongo

In [ ]:
db = MongodbHandler()

for team_name, data in team_data.items():    
    all_viz = get_all_viz(data)
    record = db.data_format(data['team_name'], data['total_matches'], data['team_market_value'], data['total_goals_2011'], data['win_percentage'], data['rain_win_percentage'], all_viz)
    db.insert_record(record)
    del(all_viz)

## Inspect Upload

In [ ]:
myclient = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
wg = myclient['summative_lab']
coll = wg['summative_collection']

In [ ]:
query_1 = coll.find({})
for x in query_1:
    print(x)